In [1]:
import wget

wget.download( "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")


'spacex_launch_dash (2).csv'

Which site has the largest successful launches?

--->> **VAFB SLC-4E** had the three largest suceeseful launches 
    
Which site has the highest launch success rate?

--->> **KSC LC-39A** it is near the sea and close to southern point in Florida, but no obvious geographic advantage vs
        CCAFS LC-40 and CCAFS SLC-40

Which payload range(s) has the highest launch success rate?

--->> **2500-5000 kg**

Which payload range(s) has the lowest launch success rate?

--->> **500kg-2000kg** given the number of failures and the small numebr of successful launches
    
Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest\ launch success rate?

--->> **FT booster** for all sites

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                    options=[
                                        {'label': 'All Sites', 'value':'ALL'},
                                        {'label': 'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                        {'label': 'VAFB SLC-4E', 'value':'VAFB SLC-4E'},
                                        {'label': 'KSC LC-39A', 'value':'KSC LC-39A'},
                                        {'label': 'CCAFS SLC-40', 'value':'CCAFS SLC-40'}],
                                    value = 'ALL', placeholder='Select Launch Site here', searchable=True),
                            
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)                            
                                dcc.RangeSlider(id='payload-slider', min = 0, max=10000, step=1000,marks={0:'0', 2000:'2000', 5000:'5000', 7000:'7000', 10000:'10000'}, value=[min_payload, max_payload]),                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"),
    Input(component_id="site-dropdown", component_property="value"))
def get_pie(value):
    if value != 'ALL':
        my_df = spacex_df[spacex_df['Launch Site'] == value].groupby(['Launch Site', 'class']).size().reset_index(name='class_count')
        
        
        pie_fig = px.pie(my_df, values='class_count', names="class", title='Total Success/Fail Launches by Site')
        return pie_fig
    else:
        pie_fig = px.pie(spacex_df, values='class', names="Launch Site", title='Total Success Launches by Site')
        return pie_fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"),
    [Input(component_id="site-dropdown", component_property="value"),
    Input(component_id="payload-slider", component_property="value")])
def get_graph(value, payload):
    if value != 'ALL':
        scat_fig = px.scatter(spacex_df, x = 'Payload Mass (kg)', y = 'class', title = 'Correlation between payload and success for all sites', color = "Booster Version Category", range_x = payload)
        return scat_fig
    else:
        data = spacex_df[spacex_df["Launch Site"] == value]
        scat_fig = px.scatter(data, x = 'Payload Mass (kg)', y = 'class', title = 'Correlation between payload and success for site %s'%value , color = "Booster Version Category", range_x = payload)
        return scat_fig

# Run the app
if __name__ == '__main__':
    app.run_server(port=8090)

Dash is running on http://127.0.0.1:8090/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8090/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1623433503.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1623433503.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_9_1m1623433503.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1623433500.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_16_0m1623433503.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56:57] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1623433503.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:56

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\dash\dash.py", line 1010, in add_context
    output_value = func(*args, **kwargs)  #

127.0.0.1 - - [11/Aug/2021 20:57:00] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [11/Aug/2021 20:57:20] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\dash\dash.py", line 1010, in add_context
    output_value = func(*args, **kwargs)  #

127.0.0.1 - - [11/Aug/2021 20:58:23] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 2070, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1515, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1513, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\flask\app.py", line 1499, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\Govindswaroop\Anaconda3\lib\site-packages\dash\dash.py", line 1010, in add_context
    output_value = func(*args, **kwargs)  #

127.0.0.1 - - [11/Aug/2021 20:58:26] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [11/Aug/2021 20:58:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [11/Aug/2021 20:58:33] "POST /_dash-update-component HTTP/1.1" 200 -
